## 2018년 05월 18일 01시 32분 11초에 추가 ##
- HHA 이미지로 학습시키기

In [1]:
%matplotlib inline
import os
import collections
import torch
import torchvision
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt

from NYUDv2Loader_HHA import *

os.environ['CUDA_VISIBLE_DEVICES'] = '2'
data_path = '/home/dongwonshin/Desktop/Datasets/NYUDv2/'
arg_string = '--arch fcn_hha --batch_size 3'

# Argument setting

In [2]:
import sys, os
import torch
import visdom
import argparse
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torch.autograd import Variable
from torch.utils import data
from tqdm import tqdm

from ptsemseg.models import get_model
from ptsemseg.loader import get_loader, get_data_path
from ptsemseg.metrics import runningScore
from ptsemseg.loss import *
from ptsemseg.augmentations import *


parser = argparse.ArgumentParser(description='Hyperparams')
parser.add_argument('--arch', nargs='?', type=str, default='fcn8s', help='Architecture to use [\'fcn8s, unet, segnet etc\']')
parser.add_argument('--img_rows', nargs='?', type=int, default=256, help='Height of the input image')
parser.add_argument('--img_cols', nargs='?', type=int, default=256, help='Width of the input image')

parser.add_argument('--img_norm', dest='img_norm', action='store_true', help='Enable input image scales normalization [0, 1] | True by default')
parser.add_argument('--no-img_norm', dest='img_norm', action='store_false', help='Disable input image scales normalization [0, 1] | True by default')
parser.set_defaults(img_norm=True)

parser.add_argument('--n_epoch', nargs='?', type=int, default=100, help='# of the epochs')
parser.add_argument('--batch_size', nargs='?', type=int, default=1, help='Batch Size')
parser.add_argument('--l_rate', nargs='?', type=float, default=1e-5, help='Learning Rate')
parser.add_argument('--feature_scale', nargs='?', type=int, default=1, help='Divider for # of features to use')
parser.add_argument('--resume', nargs='?', type=str, default=None, help='Path to previous saved model to restart from')

parser.add_argument('--visdom', dest='visdom', action='store_true', help='Enable visualization(s) on visdom | False by default')
parser.add_argument('--no-visdom', dest='visdom', action='store_false', help='Disable visualization(s) on visdom | False by default')
parser.set_defaults(visdom=False)

In [3]:
args = parser.parse_args(arg_string.split(' '))

# Setup Augmentations
data_aug= Compose([RandomRotate(10), RandomHorizontallyFlip()])

# Setup Dataloader
t_loader = NYUDv2Loader_HHA(data_path, is_transform=True)
v_loader = NYUDv2Loader_HHA(data_path, is_transform=True, split='val')

n_classes = t_loader.n_classes
trainloader = data.DataLoader(t_loader, batch_size=args.batch_size, num_workers=16, shuffle=True)
valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=16)

# Setup Metrics
running_metrics = runningScore(n_classes)

# Setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom()

    loss_window = vis.line(X=torch.zeros((1,)).cpu(),
                       Y=torch.zeros((1)).cpu(),
                       opts=dict(xlabel='minibatches',
                                 ylabel='Loss',
                                 title='Training Loss',
                                 legend=['Loss']))

# Setup Model
model = get_model(args.arch, n_classes)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()

# Check if model has custom optimizer / loss
if hasattr(model.module, 'optimizer'):
    optimizer = model.module.optimizer
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=args.l_rate, momentum=0.99, weight_decay=5e-4)

if hasattr(model.module, 'loss'):
    print('Using custom loss')
    loss_fn = model.module.loss
else:
    loss_fn = cross_entropy2d

if args.resume is not None:                                         
    if os.path.isfile(args.resume):
        print("Loading model and optimizer from checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        print("Loaded checkpoint '{}' (epoch {})"                    
              .format(args.resume, checkpoint['epoch']))
    else:
        print("No checkpoint found at '{}'".format(args.resume)) 


# Training

In [4]:
best_iou = -100.0 
for epoch in range(args.n_epoch):
    model.train()
    for i, (color_imgs, depth_imgs, label_imgs) in enumerate(trainloader):
        color_imgs = Variable(color_imgs.cuda())
        depth_imgs = Variable(depth_imgs.cuda())
        label_imgs = Variable(label_imgs.cuda())

        optimizer.zero_grad()
        outputs = model(color_imgs,depth_imgs)

        loss = loss_fn(input=outputs, target=label_imgs)

        loss.backward()
        optimizer.step()

        if args.visdom:
            vis.line(
                X=torch.ones((1, 1)).cpu() * i,
                Y=torch.Tensor([loss.data[0]]).unsqueeze(0).cpu(),
                win=loss_window,
                update='append')

        if (i+1) % 20 == 0:
            print("Epoch [%d/%d] Loss: %.4f" % (epoch+1, args.n_epoch, loss.data[0]))

    model.eval()
    for i_val, (color_images_val, depth_images_val, label_images_val) in tqdm(enumerate(valloader)):
        color_images_val = Variable(color_images_val.cuda(), volatile=True)
        depth_images_val = Variable(depth_images_val.cuda(), volatile=True)
        label_images_val = Variable(label_images_val.cuda(), volatile=True)

        outputs = model(color_images_val, depth_images_val)
        pred = outputs.data.max(1)[1].cpu().numpy()
        gt = label_images_val.data.cpu().numpy()
        running_metrics.update(gt, pred)

    score, class_iou = running_metrics.get_scores()
    for k, v in score.items():
        print(k, v)
    running_metrics.reset()

    if score['Mean IoU : \t'] >= best_iou:
        best_iou = score['Mean IoU : \t']
        state = {'epoch': epoch+1,
                 'model_state': model.state_dict(),
                 'optimizer_state' : optimizer.state_dict(),}
        torch.save(state, "../model_weights/{}_{}_best_model.pkl".format(args.arch, 'NYUDv2'))

/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1462: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")


Epoch [1/100] Loss: 2.9461
Epoch [1/100] Loss: 2.5800
Epoch [1/100] Loss: 2.2530
Epoch [1/100] Loss: 2.1810
Epoch [1/100] Loss: 1.9991
Epoch [1/100] Loss: 2.5030
Epoch [1/100] Loss: 2.1029
Epoch [1/100] Loss: 2.2210
Epoch [1/100] Loss: 1.8838
Epoch [1/100] Loss: 2.0365
Epoch [1/100] Loss: 1.6182
Epoch [1/100] Loss: 1.9812
Epoch [1/100] Loss: 1.8352


218it [00:48,  4.46it/s]


Overall Acc: 	 0.407752437922
Mean Acc : 	 0.194640308927
FreqW Acc : 	 0.235909692221
Mean IoU : 	 0.120795013148
Epoch [2/100] Loss: 1.4093
Epoch [2/100] Loss: 1.7438
Epoch [2/100] Loss: 1.8399
Epoch [2/100] Loss: 1.5092
Epoch [2/100] Loss: 1.8128
Epoch [2/100] Loss: 1.4615
Epoch [2/100] Loss: 1.7916
Epoch [2/100] Loss: 1.8657
Epoch [2/100] Loss: 1.5886
Epoch [2/100] Loss: 1.5768
Epoch [2/100] Loss: 1.5257
Epoch [2/100] Loss: 1.6277
Epoch [2/100] Loss: 1.5148


218it [00:48,  4.48it/s]


...

218it [00:49,  4.43it/s]

Overall Acc: 	 0.569915379515
Mean Acc : 	 0.428382190098
FreqW Acc : 	 0.402049483341
Mean IoU : 	 0.320547212157


Epoch [100/100] Loss: 0.3848
Epoch [100/100] Loss: 0.4147
Epoch [100/100] Loss: 0.4140
Epoch [100/100] Loss: 0.3513
Epoch [100/100] Loss: 0.2785
Epoch [100/100] Loss: 0.2577
Epoch [100/100] Loss: 0.4538
Epoch [100/100] Loss: 0.4043
Epoch [100/100] Loss: 0.3455
Epoch [100/100] Loss: 0.3329
Epoch [100/100] Loss: 0.2858
Epoch [100/100] Loss: 0.4156
Epoch [100/100] Loss: 0.3618


218it [00:49,  4.41it/s]

Overall Acc: 	 0.562834231253
Mean Acc : 	 0.424420806848
FreqW Acc : 	 0.400449627758
Mean IoU : 	 0.316227414527


# Training.. fcn_hha on NYUDv2

결과가 별로 안좋네..?
그냥 뎁스 쓴거보다.. hha쓴게 더 안좋아.. 흠...